### กลุ่ม Diabetes Prediction
1. 6610402205 นายรักษิต รุ่งรัตนไชย หมู่ 1
2. 6610402132 นายบวรรัตน์ ตั้งนรารัชชกิจ หมู่ 1
3. 6610401985 นายไชยวัตน์ หนูวัฒนา หมู่ 1

### วัตถุประสงค์ของระบบต้นแบบ

กลุ่มของพวกเราเล็งเห็นถึงความสำคัญทางด้านสุขภาพของประชาชนคนไทย ในปัจจุบันคนไทยประสบกับปัญหาสุขภาพมากมายแต่หนึ่งปัญหาที่สร้างความเสียหายให้กับสุขภาพ และคนไทยเป็นกันมากที่สุดนั่นคือโรคเบาหวาน พฤติกรรมการใช้ชีวิตของคนไทยในสมัยนี้ก็เป็นปัจจัยหนึ่งที่ทำให้ปริมาณผู้ป่วยโรคเบาหวานเพิ่มมากยิ่งขึ้น พวกเราจึงอยากทำระบบต้นแบบที่ช่วยชี้แนะหรือแนะนำผู้ที่มาใช้งานระบบต้นแบบของเราว่ามีความเสี่ยงที่จะเป็นโรคเบาหวานหรือไม่เพื่อให้เกิดการตระหนักและระมัดระวังในพฤติกรรมการใช้ชีวิตของตนเองซึ่งระบบต้นแบบของเราไม่ได้แม่นยำ 100 เปอร์เซ็นต์ ไม่สามารถใช้วินิจฉัยโรคได้แต่ใช้ประเมินความเสี่ยงเบื่องต้นของผู้ที่มาใช้งานระบบได้หากมีความเสี่ยงก็สามารถให้แพทย์วินิจัยอย่างถูกต้องและแม่นยำต่อไป

### ลิงค์ไปยังข้อมูลที่จะใช้ในระบบต้นแบบ

Link to data: https://www.kaggle.com/datasets/alexteboul/diabetes-health-indicators-dataset

In [20]:
import pandas as pd

diabetes = pd.read_csv("diabetes_binary_health_indicators_BRFSS2015.csv")

### ใช้ baseline และ วิธีการประเมินอะไรได้บ้าง อย่างไร มีค่าเป็นเท่าไหร่

แบ่งชุดข้อมูลเรียนรู้(train), ชุดข้อมูลทดสอบ(test), และชุดข้อมูลตรวจสอบ(validation)

In [21]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(diabetes, test_size = 0.3, stratify = diabetes.Diabetes_binary, random_state = 1234)
test, validation = train_test_split(test, test_size = 0.3, stratify = test.Diabetes_binary, random_state = 1234)

features = train.drop("Diabetes_binary", axis=1).columns
label = train.Diabetes_binary

In [22]:
len(train), len(test), len(validation)

(177576, 53272, 22832)

ตรวจสอบความสมดุลของข้อมูล

In [23]:
train.Diabetes_binary.value_counts()

Diabetes_binary
0.0    152834
1.0     24742
Name: count, dtype: int64

จะเห็นได้ว่าชุดข้อมูลขาดความสมดุล ดังนั้นต้องทำให้ข้อมูลสมดุลกันก่อน ด้วยการทำ Undersampling (ลดปริมาณข้อมูลของกลุ่มที่มีจำนวนเยอะ)

In [24]:
from imblearn.under_sampling import RandomUnderSampler
resampler = RandomUnderSampler(random_state=1234)

feature = diabetes.drop(columns="Diabetes_binary").columns
label = "Diabetes_binary"

train[feature], train[label] = resampler.fit_resample(train[feature], train[label])
train.dropna(inplace=True)

train.head()

,Diabetes_binary,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,HeartDiseaseorAttack,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
186497,1.0,1.0,0.0,1.0,29.0,1.0,1.0,1.0,0.0,1.0,...,1.0,1.0,5.0,2.0,6.0,1.0,1.0,13.0,3.0,2.0
18001,0.0,0.0,0.0,1.0,25.0,0.0,0.0,0.0,1.0,1.0,...,1.0,0.0,1.0,0.0,0.0,0.0,1.0,4.0,6.0,8.0
37591,0.0,0.0,0.0,1.0,25.0,1.0,0.0,0.0,0.0,1.0,...,1.0,0.0,3.0,0.0,0.0,0.0,0.0,8.0,6.0,8.0
119797,0.0,1.0,1.0,1.0,30.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,3.0,0.0,0.0,0.0,1.0,9.0,4.0,7.0
55137,1.0,1.0,0.0,1.0,37.0,1.0,0.0,0.0,1.0,0.0,...,1.0,1.0,5.0,15.0,20.0,1.0,0.0,5.0,5.0,5.0


In [25]:
train.Diabetes_binary.value_counts()

Diabetes_binary
1.0    24742
0.0    24742
Name: count, dtype: int64

สร้างโมเดล baseline จากฟีเจอร์ทั้งหมด

In [26]:
# Dummy ยังไม่ได้ทำ feature engineer
from sklearn.dummy import DummyClassifier

# baseline = DummyClassifier(strategy="uniform", random_state=1234)
baseline = DummyClassifier(strategy="stratified", random_state=1234)
# baseline = DummyClassifier(strategy="most_frequent", random_state=1234)
# baseline = DummyClassifier(strategy="constant", constant=0, random_state=1234)

baseline.fit(train[features], train.Diabetes_binary)

train_score = baseline.score(train[features], train.Diabetes_binary)
validation_score = baseline.score(validation[features], validation.Diabetes_binary) 

print(f"train score: {train_score:.3f}")
print(f"validation score: {validation_score:.3f}")

train score: 0.501
validation score: 0.502


สร้าง pipeline ของโมเดล Logistic Regression

In [27]:
from sklearn.pipeline import make_pipeline
from sklearn.compose import make_column_transformer
from sklearn.preprocessing import KBinsDiscretizer
from sklearn.linear_model import LogisticRegression

def create_pipline_logistic():
    return make_pipeline(
        make_column_transformer(
            (KBinsDiscretizer(encode="ordinal", strategy="quantile"), ["BMI"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["PhysHlth"]),
            (KBinsDiscretizer(encode="ordinal", strategy="uniform"), ["MentHlth"]),
            remainder="passthrough"
        ),
        LogisticRegression(max_iter=500)
    )

logistic_pipeline = create_pipline_logistic()
logistic_pipeline

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(remainder='passthrough',
                                   transformers=[('kbinsdiscretizer-1',
                                                  KBinsDiscretizer(encode='ordinal'),
                                                  ['BMI']),
                                                 ('kbinsdiscretizer-2',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   strategy='uniform'),
                                                  ['PhysHlth']),
                                                 ('kbinsdiscretizer-3',
                                                  KBinsDiscretizer(encode='ordinal',
                                                                   strategy='uniform'),
                                                  ['MentHlth'])])),
                ('logisticregression', LogisticRegression(max_iter=500))])

ตรวจสอบความแม่นยำของโมเดล

In [28]:
logistic_pipeline.fit(train[feature], train[label])

logistic_validation_score = logistic_pipeline.score(validation[feature], validation[label])
print(f"Logistic Validation Score: ")

0.736291170287316